In [1]:
import os

import pandas as pd
import time
import pickle
import copy

In [2]:
import disalloy
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet
from disalloy.pw_calculator import PW_calculator

In [3]:
import importlib

In [4]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [9]:
test_row = structure_DB.loc[4]
atoms_test, id_test = test_row['init_structure'].copy(), \
    test_row.name
atoms_test, id_test

(Atoms(symbols='Fe', pbc=True, cell=[[1.5, 1.5, 0.0], [0.0, 1.5, 1.5], [1.5, 0.0, 1.5]]),
 4)

In [29]:
importlib.reload(disalloy.pw_calculator)
from disalloy.pw_calculator import PW_calculator

In [30]:
pw_calc = PW_calculator()

In [31]:
pw_calc.qe_calculation(atoms_test, id_test, np=4)

2024-09-07 20:15:14 - Calculation 00000004 completed in folder calculation_00000004_5d7fe495-c30c-4e88-90e8-1690c5a762e2



[[Atoms(symbols='Fe', pbc=True, cell=[[1.5000002803596313, 1.5000002803596313, 0.0], [0.0, 1.5000002803596313, 1.5000002803596313], [1.5000002803596313, 0.0, 1.5000002803596313]]),
  -4477.328531526496,
  array([[0., 0., 0.]]),
  array([-1.94717266, -1.94717266, -1.94717266, -0.        , -0.        ,
         -0.        ])],
 [Atoms(symbols='Fe', pbc=True, cell=[[2.055624108, 2.055624108, 0.0], [0.0, 2.055624108, 2.055624108], [2.055624108, 0.0, 2.055624108]]),
  -4475.882945534491,
  array([[0., 0., 0.]]),
  array([ 0.47036664,  0.47036664,  0.47036664,  0.        ,  0.        ,
         -0.        ])],
 [Atoms(symbols='Fe', pbc=True, cell=[[1.752792014, 1.752792014, 0.0], [0.0, 1.752792014, 1.752792014], [1.752792014, 0.0, 1.752792014]]),
  -4479.299428249917,
  array([[0., 0., 0.]]),
  array([ 0.27643981,  0.27643981,  0.27643981,  0.        ,  0.        ,
         -0.        ])],
 [Atoms(symbols='Fe', pbc=True, cell=[[1.711744513, 1.711744513, 0.0], [0.0, 1.711744513, 1.711744513],

In [32]:
def relaxer_espresso(row , log_file = None):
    structure = row['init_structure']
    structure_for_relax = copy.deepcopy(structure)
    return pw_calc.qe_calculation(structure_for_relax, row.name, 
                                  log_file=log_file, np=4)

def relaxer_espresso_jitter(row , log_file = None):
    structure = row['init_structure']
    structure_for_relax = copy.deepcopy(structure)
    structure_for_relax = jitter_ic(structure_for_relax)
    structure_for_relax = jitter_ia(structure_for_relax)
    return pw_calc.qe_calculation(structure_for_relax, row.name, 
                                  log_file=log_file, np=4)

In [33]:
from disalloy.grid_run import run_packet

In [34]:
df = structure_DB.copy()
# if os.path.exists('df_espresso.pkl'):
#     df = pd.read_pickle('df_espresso.pkl')

In [35]:
relaxer_dic_espresso = {'calculator_label': 'relaxer_espresso',
                            'relaxer': relaxer_espresso}

In [36]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-07 20:22:59 - starting packet run: attempting 10 runs of size 1

2024-09-07 20:22:59 - attempting to run :
 [0, 1, 2, 3, 4, 5, 6, 7]
of size :
 8
with calculator :
 relaxer_espresso


2024-09-07 20:23:12 - Calculation 00000000 completed in folder calculation_00000000_344072ca-6f81-4b6f-b999-22019a8e0f69

2024-09-07 20:23:49 - Calculation 00000001 completed in folder calculation_00000001_ba4798ac-ff98-4580-a855-1cda6fe048c5

2024-09-07 20:24:17 - Calculation 00000002 completed in folder calculation_00000002_96bc9816-07b6-4446-a567-314d553c0c6d

2024-09-07 20:25:23 - Calculation 00000003 completed in folder calculation_00000003_6b31232d-764d-47e6-bbdd-e0ca59b368b2

2024-09-07 20:28:38 - Calculation 00000004 completed in folder calculation_00000004_fe4b0d99-598d-4750-b370-9a8c23d62fb3

2024-09-07 20:29:26 - Calculation 00000005 completed in folder calculation_00000005_7e18336c-62a7-46b6-850f-acc53cc9ee02

2024-09-07 20:30:17 - Calculation 00000006 completed in folder calculation_0

In [37]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 5, 2)
print('{}s'.format(time.time() - ti))

2024-09-08 02:05:29 - starting packet run: attempting 5 runs of size 2

2024-09-08 02:05:29 - attempting to run :
 [43, 63, 29, 8, 11]
of size :
 5
with calculator :
 relaxer_espresso


2024-09-08 02:09:33 - Calculation 00000043 completed in folder calculation_00000043_d4ed1427-cd96-4bfe-846c-d93343deb0ea

2024-09-08 02:13:21 - Calculation 00000063 completed in folder calculation_00000063_548516af-1cad-4d83-9ce3-15fad547e4ec

2024-09-08 02:22:27 - Calculation 00000029 completed in folder calculation_00000029_c41c4825-4c68-4a0f-8a6d-c6386292ef8e

2024-09-08 02:28:21 - Calculation 00000008 completed in folder calculation_00000008_77a703dd-daad-4d87-9492-436786afccde

2024-09-08 02:36:00 - Calculation 00000011 completed in folder calculation_00000011_0718d8df-9e00-4a28-9871-ddd78b007c8a

2024-09-08 02:36:54 - Succesfuly ran packet of 5 runs in 1830.9920003414154s

1885.686565876007s


In [43]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 5, 2)
print('{}s'.format(time.time() - ti))

2024-09-08 02:42:33 - starting packet run: attempting 5 runs of size 2

2024-09-08 02:42:34 - attempting to run :
 [58, 23, 33, 36, 39]
of size :
 5
with calculator :
 relaxer_espresso


2024-09-08 02:47:58 - Calculation 00000058 completed in folder calculation_00000058_16c9b313-d4c4-4ec0-a09f-59d5df552087

2024-09-08 02:57:51 - Calculation 00000023 completed in folder calculation_00000023_c29114c6-a81b-4320-98a9-68dac8abe887

2024-09-08 03:09:52 - Calculation 00000033 completed in folder calculation_00000033_d8213e22-988c-4d88-b5a2-89ab15a5a89c

2024-09-08 03:13:59 - Calculation 00000036 completed in folder calculation_00000036_e77ac35d-817a-4ceb-a39a-9ee0c14cf57e

2024-09-08 03:46:14 - Calculation 00000039 completed in folder calculation_00000039_f0e4ebd8-8c6b-42f7-a492-af8baec8b637

2024-09-08 03:46:35 - Succesfuly ran packet of 5 runs in 3820.4080691337585s

3841.2541711330414s


In [44]:
df[df[relaxer_dic_espresso['calculator_label']].notnull()]

,init_structure,size,relaxer_espresso
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Al', [0.0, 0.0, 0.0], index=0)), -531..."
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Co', [0.0, 0.0, 0.0], index=0)), -405..."
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Cr', [0.0, 0.0, 0.0], index=0)), -238..."
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Cu', [0.0, 0.0, 0.0], index=0)), -289..."
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Fe', [0.0, 0.0, 0.0], index=0)), -447..."
5,"(Atom('Mn', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Mn', [0.0, 0.0, 0.0], index=0)), -287..."
6,"(Atom('Ni', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Ni', [0.0, 0.0, 0.0], index=0)), -466..."
7,"(Atom('V', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('V', [0.0, 0.0, 0.0], index=0)), -1967..."
8,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"[[(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom(..."
11,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('F...",2,"[[(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom(..."


In [45]:
for i in df.loc[:7, 'init_structure']:
    print(i.cell.array)

[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]


In [46]:
calculated_index = df[df[relaxer_dic_espresso[
    'calculator_label']].notnull()].index

In [47]:
for i, j  in zip(
    df.loc[calculated_index, 'init_structure'], 
    df.loc[calculated_index, 'relaxer_espresso'].array):
    print(i.get_chemical_symbols())
    print(i.cell.array)
    print()
    print(j[-1][0].cell.array)
    
    print()
    if False:
        print('ijs')
        for ij in j:
            print('ij')
            print(ij[0].cell.array)
            print(ij[1])
            print(ij[2])
            print(ij[3])
            print('@')
            print()
    print(compute_distortion(i, j[-1][0]))
    print()
    print('#'*50)

['Al']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[ 2.02154522  2.02154522 -0.        ]
 [-0.          2.02154522  2.02154522]
 [ 2.02154522 -0.          2.02154522]]

4.272219962348625e-16

##################################################
['Co']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[1.72334563 1.72334563 0.        ]
 [0.         1.72334563 1.72334563]
 [1.72334563 0.         1.72334563]]

1.7281680248448638e-16

##################################################
['Cr']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[ 1.81063764  1.81063764 -0.        ]
 [-0.          1.81063764  1.81063764]
 [ 1.81063764  0.          1.81063764]]

1.7281680248448638e-16

##################################################
['Cu']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[ 1.80981888  1.80981888 -0.        ]
 [-0.          1.80981888  1.80981888]
 [ 1.80981888 -0.          1.80981888]]

4.272219962348625e-16

##################################################
['Fe']
[[1.5 1.5 0.

In [41]:
relaxer_dic_espresso_jitter = {
    'calculator_label': 'relaxer_espresso_jitter',
    'relaxer': relaxer_espresso_jitter}

In [42]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter, 10, 1,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-06 12:57:28 - starting packet run: attempting 10 runs of size 1

2024-09-06 13:32:41 - index of ran strctures:
 [0, 1, 2, 4, 5, 6, 7]

2024-09-06 13:32:41 - Succesfuly ran packet of 7 runs in 2086.1792826652527s

2113.194128036499s


In [43]:
calculated_index = df[df[relaxer_dic_espresso_jitter[
    'calculator_label']].notnull()].index

In [33]:
for i, j, k  in zip(
    df.loc[calculated_index, 'init_structure'], 
    df.loc[calculated_index, 'relaxer_espresso'].array,
    df.loc[calculated_index, 'relaxer_espresso_jitter'].array):
    print(i.get_chemical_symbols())
    print(i.cell.array)
    print()
    print(j[-1][0].cell.array)
    print(compute_distortion(i, j[-1][0]))
    print()
    print(k[-1][0].cell.array)
    print(compute_distortion(i, k[-1][0]))
    
    print()
    if False:
        print('ijs')
        for ij in j:
            print('ij')
            print(ij[0].cell.array)
            print(ij[1])
            print(ij[2])
            print(ij[3])
            print('@')
            print()
    print()
    print('#'*50)

['Cu']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[1.80982878 1.80982878 0.        ]
 [0.         1.80982878 1.80982878]
 [1.80982878 0.         1.80982878]]
1.7281680248448638e-16

[[ 1.81194863e+00  1.80601090e+00  5.24523000e-04]
 [ 4.67841200e-03  1.80780547e+00  1.81316908e+00]
 [ 1.81021323e+00 -4.56658900e-03  1.81005496e+00]]
0.0011047402690813656


##################################################


In [34]:
for i in df.loc[:7, 'init_structure']:
    print(i.cell.array)

[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
